In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

st.title('워드 임베딩 프로젝트')

DATE_COLUMN = 'date/time'
DATA_URL = ('https://s3-us-west-2.amazonaws.com/'
            'streamlit-demo-data/uber-raw-data-sep14.csv.gz')

@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data

data_load_state = st.text('Loading data...')
data = load_data(10000)
data_load_state.text("Done! (using st.cache)")

# if st.button("버튼 클릭"):
#     st.write("Data Loading..")
#     # 버튼을 누르면 동작하게 하고 함수 추가

# checkbox_btn1 = st.checkbox('옵션1')
# checkbox_btn2 = st.checkbox('옵션2')

# if checkbox_btn1:
#     st.write('Great!')

# if checkbox_btn2:
#     st.write('Good!')

# if st.checkbox('Show raw data'):
#     st.subheader('Raw data')
#     st.write(data)

# multi_select = st.multiselect('Please select somethings in multi selectbox!',
#                             ['A', 'B', 'C', 'D'])

# st.write('You selected:', multi_select)

# st.subheader('Number of pickups by hour')
# hist_values = np.histogram(data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]
# st.bar_chart(hist_values)

# hour_to_filter = st.slider('hour', 0, 23, 17)
# filtered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]

# st.subheader('Map of all pickups at %s:00' % hour_to_filter)
# st.map(filtered_data)

################################

# 파일 경로
TENSOR_FILE = "ko_w2v_tensor_asmr.tsv"
META_FILE = "ko_w2v_metadata_asmr.tsv"

@st.cache_data
def load_model():
    # 메타데이터 (단어 리스트)
    words = pd.read_csv(META_FILE, sep="\t", header=None)[0].tolist()

    # 텐서 (벡터 리스트)
    vectors = np.loadtxt(TENSOR_FILE, delimiter="\t")

    # 단어-벡터 딕셔너리로 만들기
    word_to_vec = {word: vec for word, vec in zip(words, vectors)}
    return word_to_vec, words, vectors

word_to_vec, words, vectors = load_model()

# UI
st.title("🔍 Word2Vec 유사 단어 검색기 (ASMR 댓글 기반)")
query = st.text_input("단어를 입력하세요:")

if query:
    if query not in word_to_vec:
        st.error("해당 단어는 단어장에 없습니다.")
    else:
        query_vec = word_to_vec[query].reshape(1, -1)
        sims = cosine_similarity(query_vec, vectors)[0]
        top_n = 10

        top_idx = sims.argsort()[::-1][1:top_n+1]  # 자기 자신 제외
        st.write("유사한 단어들:")
        for i in top_idx:
            st.write(f"{words[i]}: {sims[i]:.4f}")

# cosine_similarity

st.markdown("---")
st.subheader("자유로운 단어 연산 (예: 서울 - 한국 + 일본 + 음식 - 배고픔)")

expression = st.text_input("단어 연산식", "")

if expression:
    try:
        tokens = expression.strip().split()

        positive = []
        negative = []
        op = "+"  # 기본은 더하기

        for token in tokens:
            if token in ["+", "-"]:
                op = token
            else:
                if token not in word_to_vec:
                    st.error(f"단어 '{token}'가 단어장에 없습니다.")
                    break
                if op == "+":
                    positive.append(token)
                elif op == "-":
                    negative.append(token)

        if positive:
            vec_result = sum([word_to_vec[w] for w in positive])
            if negative:
                vec_result -= sum([word_to_vec[w] for w in negative])
            vec_result = vec_result.reshape(1, -1)

            sims = cosine_similarity(vec_result, vectors)[0]
            top_idx = sims.argsort()[::-1][:5]

            st.write(f"연산 결과:")
            for i in top_idx:
                st.write(f"{words[i]}: {sims[i]:.4f}")
        else:
            st.warning("➕에 해당하는 단어가 1개 이상 있어야 해요.")

    except Exception as e:
        st.error(f"에러 발생: {e}")


